In [139]:
# Clears lingering variables from Jupyter
from IPython import get_ipython
get_ipython().magic('reset -sf')

# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# returns array with 0th index as min, 1th index as max
def findMinMaxOfSequence(list, n):
    min_max = []

    s = set()
    for i in range(n):
        s.add(list[i])

    listMin = n
    listMax = 0
    for i in range(n):
        if s.__contains__(list[i]):
            j = list[i]
            while (s.__contains__(j)):
                j += 1

            listMax = max(listMax, j - list[i])
            listMin = min(listMin, j - list[i])

    min_max = [listMin, listMax]

    return min_max

# Import data and name columns
data = pd.read_csv('both.csv', header=None)
data.columns = ['packet_id', 'max_packets', 'ip', 'port']

# split data by client (same IP + same Port)
groupedBy = data.groupby(['port', 'ip'], sort=False)

# extract keys from groups
keys = groupedBy.groups.keys()

# aggregate variables
total_missing = 0
min_lost = 0
max_lost = 0
min_unordered = 0
max_unordered = 0


# process groups  
for i in keys:
    print('client: ' + str(i))
    group = groupedBy.get_group(i)
    group.reset_index(drop=True, inplace=True)
    print(group)
    print ('\n')
    packetsReceived = len(group.index)
    packetsIntended = group.iloc[1]['max_packets']
    packetsLost = packetsIntended - packetsReceived
    total_missing += packetsLost
    print('num packets received: ' + str(packetsReceived))
    print('num packets intended: ' + str(packetsIntended))
    print('packets lost: ' + str(packetsLost))

    # make a series for faster processing
    receivedSeries = pd.Series(data = group['packet_id'], index=list(range(0, packetsIntended)))
    expectedSeries = pd.Series(range(1, packetsIntended + 1))
    expectedSeries.rename('packet_id')

    # find missing and out of order packets
    lastVal = 0
    val = 0
    missing = []
    out_of_order = []
    visited = []
    full = list(range(1, packetsIntended + 1))

    for index, value in receivedSeries.items():
        lastVal = val
        val = value

        visited.append(val)

        # found a number previously marked as missing
        # if val in missing:
        #     missing.remove(val)
        
        # found 
        # elif val != lastVal + 1 and lastVal + 1 < packetsIntended and val not in visited:
        #     missing.append(lastVal + 1)

        # found out of order number
        if val != lastVal + 1 and np.isnan(val) == False:
            out_of_order.append(val)

    out_of_order.sort()

    # print packets lost and out of order
    if packetsLost != 0:
        missing = list(set(full).difference(set(visited)))
        print('lost packet ids: ' + str(missing))
    print('out of order packet ids: ' + str(out_of_order))

    # sequence min/maxes
    out_of_order_min_max = findMinMaxOfSequence(out_of_order, len(out_of_order))
    print('min # packets out of order in sequence: ' + str(out_of_order_min_max[0]))
    print('max # packets out of order in sequence: ' + str(out_of_order_min_max[1]))
    lost_min_max = findMinMaxOfSequence(missing, len(missing))
    print('min # packets lost in sequence: ' + str(lost_min_max[0]))
    print('max # packets lost in sequence: ' + str(lost_min_max[1]))

    # debug only: compare function for verification
    comparison = receivedSeries.compare(expectedSeries)
    if len(comparison) != 0:
        print('\nself = received, other = expected')
        print(comparison)

    ## update aggregate variables
    


    print('\n\n')

    

average_missing = round(total_missing / groupedBy.ngroups, 2)
print('average lost: ' + str(average_missing))






client: (9000, '1.0.0.0')
    packet_id  max_packets       ip  port
0           1          100  1.0.0.0  9000
1           2          100  1.0.0.0  9000
2           3          100  1.0.0.0  9000
3           4          100  1.0.0.0  9000
4           5          100  1.0.0.0  9000
..        ...          ...      ...   ...
92         93          100  1.0.0.0  9000
93         94          100  1.0.0.0  9000
94         95          100  1.0.0.0  9000
95         96          100  1.0.0.0  9000
96        100          100  1.0.0.0  9000

[97 rows x 4 columns]


num packets received: 97
num packets intended: 100
packets lost: 3
lost packet ids: [97, 98, 99]
out of order packet ids: [82.0, 84.0, 87.0, 88.0, 89.0, 90.0, 91.0, 100.0]
min # packets out of order in sequence:1.0
max # packets out of order in sequence:5.0
min # packets lost in sequence:1
max # packets lost in sequence:3

self = received, other = expected
     self  other
81   84.0   82.0
82   85.0   83.0
83   86.0   84.0
84   82.0   85.0
8